In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
class Model (nn.Module):
    def __init__(self, iLayers=146, h1=72, h2=36, h3=16, h4=8, oLayer=1):
        super().__init__()
        self.fc1 = nn.Linear(iLayers, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.fc3 = nn.Linear(h2, h3)
        self.fc4 = nn.Linear(h3, h4)
        self.out = nn.Linear(h4, oLayer)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.out(x)

        return x

In [3]:
torch.manual_seed(16)
model = Model()

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [5]:
stats_24 = pd.read_csv('Data Set/21-24Stats.csv')
matchups_24 = pd.read_csv('Data Set/24Matches.csv')

In [6]:
y = matchups_24['SCORE_DIF'].values.reshape(-1,1)
x = matchups_24.merge(stats_24, left_on='TEAM_0_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.merge(stats_24, left_on='TEAM_1_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.drop(['MATCH_ID', 'TEAM_0', 'TEAM_1', 'TEAM_0_ID', 'TEAM_1_ID', 'TEAM_ID_0', 'TEAM_ID_1', 'SCORE_DIF'], axis=1)

scaler = StandardScaler()
x = scaler.fit_transform(x)
    
# Remove NaNs and Infs from the data
x = torch.nan_to_num(torch.tensor(x, dtype=torch.float32))
y = torch.nan_to_num(torch.tensor(y, dtype=torch.float32))

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=16)

In [8]:
criterion = nn.MSELoss()
# if error doesn't go down after a bunch of iterations (epochs), lower our learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)

In [9]:
epochs = 600
losses = []
for i in range(epochs):
  # Go forward and get a prediction
  y_pred = model(x_train) # Get predicted results

  loss = criterion(y_pred, y_train)
  loss = loss / len(x_train) 

  # print every 10 epoch
  if i % 10 == 0:
    print(f'Epoch: {i} and loss: {loss}')

  # Do some back propagation: take the error rate of forward propagation and feed it back
  # thru the network to fine tune the weights
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

model.eval()
with torch.no_grad():
    test_pred = model(x_test)
    test_loss = criterion(test_pred, y_test) / len(x_test)
    print(f'Test Loss: {test_loss:.4f}')

Epoch: 0 and loss: 1.9268829822540283
Epoch: 10 and loss: 1.9264332056045532
Epoch: 20 and loss: 1.9260013103485107
Epoch: 30 and loss: 1.9255636930465698
Epoch: 40 and loss: 1.925047755241394
Epoch: 50 and loss: 1.9243937730789185
Epoch: 60 and loss: 1.9235877990722656
Epoch: 70 and loss: 1.9225722551345825
Epoch: 80 and loss: 1.9212515354156494
Epoch: 90 and loss: 1.9196399450302124
Epoch: 100 and loss: 1.917697787284851
Epoch: 110 and loss: 1.9153530597686768
Epoch: 120 and loss: 1.9124780893325806
Epoch: 130 and loss: 1.9089579582214355
Epoch: 140 and loss: 1.9046109914779663
Epoch: 150 and loss: 1.899287223815918
Epoch: 160 and loss: 1.8927927017211914
Epoch: 170 and loss: 1.88491690158844
Epoch: 180 and loss: 1.875443696975708
Epoch: 190 and loss: 1.8640830516815186
Epoch: 200 and loss: 1.8505805730819702
Epoch: 210 and loss: 1.8347023725509644
Epoch: 220 and loss: 1.81623375415802
Epoch: 230 and loss: 1.7948968410491943
Epoch: 240 and loss: 1.7704503536224365
Epoch: 250 and loss

In [10]:
stats_25 = pd.read_csv('Data Set/22-25Stats.csv')
matchups_25 = pd.read_csv('Data Set/25Matches.csv')

In [11]:
y = matchups_25['SCORE_DIF'].values.reshape(-1,1)
x = matchups_25.merge(stats_25, left_on='TEAM_0_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.merge(stats_25, left_on='TEAM_1_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.drop(['MATCH_ID', 'TEAM_0', 'TEAM_1', 'TEAM_0_ID', 'TEAM_1_ID', 'TEAM_ID_0', 'TEAM_ID_1', 'SCORE_DIF'], axis=1)

scaler = StandardScaler()
x = scaler.fit_transform(x)
    
# Remove NaNs and Infs from the data
x = torch.nan_to_num(torch.tensor(x, dtype=torch.float32))
y = torch.nan_to_num(torch.tensor(y, dtype=torch.float32))

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=16)

In [13]:
epochs = 200
losses = []
for i in range(epochs):
  # Go forward and get a prediction
  y_pred = model(x_train) # Get predicted results

  loss = criterion(y_pred, y_train)
  loss = loss / len(x_train) 

  # print every 10 epoch
  if i % 10 == 0:
    print(f'Epoch: {i} and loss: {loss}')

  # Do some back propagation: take the error rate of forward propagation and feed it back
  # thru the network to fine tune the weights
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

model.eval()
with torch.no_grad():
    test_pred = model(x_test)
    test_loss = criterion(test_pred, y_test) / len(x_test)
    print(f'Test Loss: {test_loss:.4f}')

Epoch: 0 and loss: 1.5790315866470337
Epoch: 10 and loss: 1.468571662902832
Epoch: 20 and loss: 1.3566924333572388
Epoch: 30 and loss: 1.2917975187301636
Epoch: 40 and loss: 1.2552143335342407
Epoch: 50 and loss: 1.2269881963729858
Epoch: 60 and loss: 1.2029179334640503
Epoch: 70 and loss: 1.1813461780548096
Epoch: 80 and loss: 1.1612091064453125
Epoch: 90 and loss: 1.1417486667633057
Epoch: 100 and loss: 1.1228500604629517
Epoch: 110 and loss: 1.1045985221862793
Epoch: 120 and loss: 1.0865333080291748
Epoch: 130 and loss: 1.0689526796340942
Epoch: 140 and loss: 1.051456093788147
Epoch: 150 and loss: 1.0342626571655273
Epoch: 160 and loss: 1.0170843601226807
Epoch: 170 and loss: 0.9997708797454834
Epoch: 180 and loss: 0.982223391532898
Epoch: 190 and loss: 0.9644133448600769
Test Loss: 3.2941


In [14]:
x = pd.read_csv('Data Set/R64.csv')
x = x.merge(stats_25, left_on='TEAM_0_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.merge(stats_25, left_on='TEAM_1_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.drop(['MATCH_ID', 'TEAM_0', 'TEAM_1', 'TEAM_0_ID', 'TEAM_1_ID', 'TEAM_ID_0', 'TEAM_ID_1'], axis=1)

x = scaler.fit_transform(x)
x = torch.nan_to_num(torch.tensor(x, dtype=torch.float32))

In [ ]:
model.eval()
with torch.no_grad():
    test_pred = model(x)
    pred_winners = ['Team 1' if score > 0 else 'Team 2' for score in test_pred]
    for i, winner in enumerate(pred_winners):
        print(f"Match {i+1}: {winner} | +{np.ceil(abs(test_pred[i].item())): } Points")

Match 1: Team 1 | + 17.0 Points
Match 2: Team 2 | + 5.0 Points
Match 3: Team 2 | + 2.0 Points
Match 4: Team 2 | + 1.0 Points
Match 5: Team 2 | + 3.0 Points
Match 6: Team 1 | + 4.0 Points
Match 7: Team 2 | + 1.0 Points
Match 8: Team 1 | + 4.0 Points
Match 9: Team 1 | + 7.0 Points
Match 10: Team 2 | + 6.0 Points
Match 11: Team 2 | + 3.0 Points
Match 12: Team 2 | + 4.0 Points
Match 13: Team 2 | + 2.0 Points
Match 14: Team 1 | + 2.0 Points
Match 15: Team 2 | + 4.0 Points
Match 16: Team 1 | + 5.0 Points
Match 17: Team 1 | + 12.0 Points
Match 18: Team 2 | + 4.0 Points
Match 19: Team 2 | + 2.0 Points
Match 20: Team 2 | + 5.0 Points
Match 21: Team 2 | + 5.0 Points
Match 22: Team 1 | + 3.0 Points
Match 23: Team 2 | + 4.0 Points
Match 24: Team 1 | + 2.0 Points
Match 25: Team 1 | + 18.0 Points
Match 26: Team 2 | + 4.0 Points
Match 27: Team 2 | + 2.0 Points
Match 28: Team 2 | + 6.0 Points
Match 29: Team 2 | + 5.0 Points
Match 30: Team 1 | + 1.0 Points
Match 31: Team 2 | + 2.0 Points
Match 32: Team

In [27]:
x = pd.read_csv('Data Set/R32.csv')
x = x.merge(stats_25, left_on='TEAM_0_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.merge(stats_25, left_on='TEAM_1_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.drop(['MATCH_ID', 'TEAM_0', 'TEAM_1', 'TEAM_0_ID', 'TEAM_1_ID', 'TEAM_ID_0', 'TEAM_ID_1'], axis=1)

x = scaler.fit_transform(x)
x = torch.nan_to_num(torch.tensor(x, dtype=torch.float32))

In [ ]:
model.eval()
with torch.no_grad():
    test_pred = model(x)
    pred_winners = ['Team 1' if score > 0 else 'Team 2' for score in test_pred]
    for i, winner in enumerate(pred_winners):
        print(f"Match {i+1}: {winner} | +{np.ceil(abs(test_pred[i].item())): } Points")

Match 1: Team 1 | + 5.0 Points
Match 2: Team 2 | + 2.0 Points
Match 3: Team 2 | + 2.0 Points
Match 4: Team 1 | + 4.0 Points
Match 5: Team 1 | + 6.0 Points
Match 6: Team 2 | + 2.0 Points
Match 7: Team 2 | + 3.0 Points
Match 8: Team 2 | + 2.0 Points
Match 9: Team 2 | + 5.0 Points
Match 10: Team 2 | + 3.0 Points
Match 11: Team 2 | + 3.0 Points
Match 12: Team 2 | + 4.0 Points
Match 13: Team 1 | + 13.0 Points
Match 14: Team 1 | + 1.0 Points
Match 15: Team 2 | + 3.0 Points
Match 16: Team 2 | + 4.0 Points


In [29]:
x = pd.read_csv('Data Set/S16.csv')
x = x.merge(stats_25, left_on='TEAM_0_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.merge(stats_25, left_on='TEAM_1_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.drop(['MATCH_ID', 'TEAM_0', 'TEAM_1', 'TEAM_0_ID', 'TEAM_1_ID', 'TEAM_ID_0', 'TEAM_ID_1'], axis=1)

x = scaler.fit_transform(x)
x = torch.nan_to_num(torch.tensor(x, dtype=torch.float32))

In [ ]:
model.eval()
with torch.no_grad():
    test_pred = model(x)
    pred_winners = ['Team 1' if score > 0 else 'Team 2' for score in test_pred]
    for i, winner in enumerate(pred_winners):
        print(f"Match {i+1}: {winner} | +{np.ceil(abs(test_pred[i].item())): } Points")

Match 1: Team 1 | + 8.0 Points
Match 2: Team 1 | + 4.0 Points
Match 3: Team 1 | + 3.0 Points
Match 4: Team 2 | + 3.0 Points
Match 5: Team 2 | + 2.0 Points
Match 6: Team 2 | + 4.0 Points
Match 7: Team 1 | + 6.0 Points
Match 8: Team 2 | + 5.0 Points


In [31]:
x = pd.read_csv('Data Set/E8.csv')
x = x.merge(stats_25, left_on='TEAM_0_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.merge(stats_25, left_on='TEAM_1_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.drop(['MATCH_ID', 'TEAM_0', 'TEAM_1', 'TEAM_0_ID', 'TEAM_1_ID', 'TEAM_ID_0', 'TEAM_ID_1'], axis=1)

x = scaler.fit_transform(x)
x = torch.nan_to_num(torch.tensor(x, dtype=torch.float32))

In [ ]:
model.eval()
with torch.no_grad():
    test_pred = model(x)
    pred_winners = ['Team 1' if score > 0 else 'Team 2' for score in test_pred]
    for i, winner in enumerate(pred_winners):
        print(f"Match {i+1}: {winner} | +{np.ceil(abs(test_pred[i].item())): } Points")

Match 1: Team 1 | + 5.0 Points
Match 2: Team 1 | + 3.0 Points
Match 3: Team 2 | + 2.0 Points
Match 4: Team 2 | + 5.0 Points


In [33]:
x = pd.read_csv('Data Set/F4.csv')
x = x.merge(stats_25, left_on='TEAM_0_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.merge(stats_25, left_on='TEAM_1_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.drop(['MATCH_ID', 'TEAM_0', 'TEAM_1', 'TEAM_0_ID', 'TEAM_1_ID', 'TEAM_ID_0', 'TEAM_ID_1'], axis=1)

x = scaler.fit_transform(x)
x = torch.nan_to_num(torch.tensor(x, dtype=torch.float32))

In [ ]:
model.eval()
with torch.no_grad():
    test_pred = model(x)
    pred_winners = ['Team 1' if score > 0 else 'Team 2' for score in test_pred]
    for i, winner in enumerate(pred_winners):
        print(f"Match {i+1}: {winner} | +{np.ceil(abs(test_pred[i].item())): } Points")

Match 1: Team 1 | + 1.0 Points
Match 2: Team 2 | + 3.0 Points


In [35]:
x = pd.read_csv('Data Set/Champ.csv')
x = x.merge(stats_25, left_on='TEAM_0_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.merge(stats_25, left_on='TEAM_1_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.drop(['MATCH_ID', 'TEAM_0', 'TEAM_1', 'TEAM_0_ID', 'TEAM_1_ID', 'TEAM_ID_0', 'TEAM_ID_1'], axis=1)

x = scaler.fit_transform(x)
x = torch.nan_to_num(torch.tensor(x, dtype=torch.float32))

In [ ]:
model.eval()
with torch.no_grad():
    test_pred = model(x)
    pred_winners = ['Team 1' if score > 0 else 'Team 2' for score in test_pred]
    for i, winner in enumerate(pred_winners):
        print(f"Match {i+1}: {winner} | +{np.ceil(abs(test_pred[i].item())): } Points")

Match 1: Team 2 | + 1.0 Points
